In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  7 15:07:54 2020

@author: yubim
"""

# Functional API test file


'''
Notes :
    TimeDistributed : create model that process sequence of inputs

'''

'''
Additional notes :
    When encoder is traine completely, merged model exhibits much better
    performance rather than training decoder with more epochs.
'''


'''
Variables :
    encoder_h : h^<N>, encoder hidden state(maps to internal state of LSTM, C)
    decoder_h : decoder hidden state(maps to internal state of LSTM, C)

'''
%tensorflow_version 2.x
import os
from os import listdir
from os.path import isfile, join

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Lambda, Activation, Dropout

import numpy as np
import pandas as pd

import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.metrics import mean_absolute_percentage_error
from tensorflow.keras import regularizers

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Parameters
from tensorflow import concat
from tensorflow import broadcast_to


import tensorflow as tf

import matplotlib

tf.test.gpu_device_name()

with tf.device('/device:GPU:0'):
  # Different configurations for different computation structures
  # gpu = input('On Server?(y/n) : ')
  gpu = 'n'
  if(gpu == 'y'):
      # Ask for the parameters
      encoderEpoch    = 10
      decoderEpoch    = 5
      numNeuron       = 128
      batchSize       = 1

      forecastHorizon = input('T - M         : ')
      directoryName   = input('Directory     : ') # directory for the output plots (without slash, Example : test10-1)
      dataSize        = input('Data size     : ')
      M               = input('M             : ')


      matplotlib.use('pdf')

      forecastHorizon = int(forecastHorizon)
      M               = int(M)
      dataSize        = int(dataSize)

      T = M + forecastHorizon

      stateful = True


      # sys.stdout = open(directoryName + '/' + 'log.txt', "w")
  else:
      # Ask for the parameters
      encoderEpoch    = 50
      decoderEpoch    = 300
      numNeuron       = 512
      batchSize       = 1

      forecastHorizon = '24'
      directoryName   = '/content/drive/My Drive/Colab Notebooks/load' # directory for the output plots (without slash, Example : test10-1)
      dataSize        = '1440'
      M               = '24'


      matplotlib.use('pdf')

      forecastHorizon = int(forecastHorizon)
      M               = int(M)
      dataSize        = int(dataSize)

      T = M + forecastHorizon



      stateful = False



      os.environ['CUDA_VISIBLE_DEVICES']="-1"

  import matplotlib.pyplot as plt




  #################### Code starts
  # Parameters for LSTM input (we need to know the input shape)
  # These are dynamic (2 month will not yield OneHotEncoding with 12 length)
  num_months = 0
  num_days   = 0
  num_hours  = 0

  def plot(figNum, originalData, lstmOuts, customTag=''):
      # lstmOuts -> predictions inverse transformed
      # originalData -> data read from file

      # forecastLong : forecast with given (T - M)
      forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      listStart = -50
      listEnd   = -2
      plt.plot(originalData[listStart:listEnd])
      plt.plot(forecastLong[listStart:listEnd])
      plt.plot(persis[listStart+1:listEnd+1])
      plt.plot(startingPoints[listStart:listEnd], 'rX')

      title = ''
      title = title + customTag + '\n'
      title = title + ('numNeuron = ' + str(numNeuron) + ', ')
      title = title + ('dataSize = ' + str(dataSize) + ', ')
      title = title + ('M = ' + str(M) + ', ')
      title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
      title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
      # title = title + ('loss = ' + str(loss) + ', ')
      # title = title + ('optimizer = ' + str(optimizer) + ', ')
      title = title + ('(T - M) = ' + str(T-M) + ', ')
      title = title + ('currentEpoch = ' + str(figNum) + '')
      plt.title(title, fontsize=6)

      plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
      plt.legend(['Original Data', 'LSTM', 'Persistance', 'First Forecast'])
      plt.savefig(directoryName + '/' + customTag + str(figNum) + '.png', bbox_inches="tight")
      plt.cla()

  def plotTest(figNum, originalData, lstmOuts, customTag=''):
      # lstmOuts -> predictions inverse transformed
      # originalData -> data read from file

      # forecastLong : forecast with given (T - M)
      forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      listStart = -50
      listEnd   = -2
      plt.plot(originalData[listStart:listEnd])
      plt.plot(forecastLong[listStart:listEnd])
      plt.plot(persis[listStart+1:listEnd+1])
      plt.plot(startingPoints[listStart:listEnd], 'rX')

      title = ''
      title = title + customTag + '\n'
      title = title + ('numNeuron = ' + str(numNeuron) + ', ')
      title = title + ('dataSize = ' + str(dataSize) + ', ')
      title = title + ('M = ' + str(M) + ', ')
      title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
      title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
      # title = title + ('loss = ' + str(loss) + ', ')
      # title = title + ('optimizer = ' + str(optimizer) + ', ')
      title = title + ('(T - M) = ' + str(T-M) + ', ')
      title = title + ('currentEpoch = ' + str(figNum) + '')
      plt.title(title, fontsize=6)

      plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
      plt.legend(['Original Data', 'LSTM', 'Persistance', 'First Forecast'])
      plt.savefig(directoryName + '/' + customTag + str(figNum) + '_test.png', bbox_inches="tight")
      plt.cla()


  def noneNum(allorders):
      # Replace None's in a python list with 0's and return the output
      allorders = list(allorders)
      output = [i if i is not None else 0 for i in allorders]

      return output

  def calculateError(originalData, lstmOuts):
      forecastLong   = [lstmOuts[i//(T - M)*(T - M)][i%(T - M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      # Calculate error
      endIndex = -(T - M) + 1
      if endIndex == 0:
          endIndex = originalData.shape[0]

      text = ''

      maemym  = mean_absolute_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      maepers = mean_absolute_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      text = text + "MAE my model     : " + str(maemym)
      text = text + '\n'
      text = text + "MAE persistence  : " + str(maepers)
      text = text + '\n'
      text = text + '\n'

      msemym  = mean_squared_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      msepers = mean_squared_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      text = text + "MSE my model     : " + str(msemym)
      text = text + '\n'
      text = text + "MSE persistence  : " + str(msepers)
      text = text + '\n'
      text = text + '\n'

      # # TODO - can be replaced by tensorflow
      # mapemym  = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      # text = text + "MAPE my model    : " + str(mapemym)
      # text = text + '\n'
      # text = text + "MAPE persistence : " + str(mapepers)
      # text = text + '\n'
      # text = text + '\n'
      
      # mapemym = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      # with tensorflow.Session() as sess:
          # mapemym_val = mapemym.eval()
          # mapepers_val = mapepers.eval()
      
      # text = text + "MAPE my model    : " + str(mapemym_val)
      # text = text + '\n'
      # text = text + "MAPE persistence : " + str(mapepers_val)
      # text = text + '\n'
      # text = text + '\n'


      return text

  #def mean_absolute_percentage_error(y_true, y_pred):
  #    # Not implemented in sklearn
  #
  #    # Not recommended to use since negative values
  #    # create negative losses
  #
  #    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  def readData(dataType):
      if dataType == 'train':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_train.csv',sep=',')
      elif dataType == 'valid':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_valid.csv',sep=',')
      elif dataType == 'test':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_test.csv',sep=',')


      outputDat = np.array(data.total_consumption.iloc[:len(data)])
      date_time = [datetime.datetime.strptime(data.timestamp.iloc[i], '%Y-%m-%d %H:%M:%S') for i in range(1,len(data))]

      return outputDat, date_time


  # Taken from test3.py
  # Taken from test3.py
  def prepareDataTraining(Xa, date_time):
      """
      Data Representation :
      [samples, timesteps, features]

      Each timestep :
      [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
      Example :
      [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

      Each sample :
      [ timestep_0, timestep_1, ..., timestep_n ]

      Overall(3D) :
      [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
      """
      
      dataSize = len(Xa)

      scaler = MinMaxScaler(feature_range =(-1,1))
      X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])
      
  #    global num_hours
  #    global num_days
  #    global num_months
      
      # Prepare features
      all_hours  = [o.hour      for o in date_time]
      all_days   = [o.weekday() for o in date_time]
  #    all_months = [o.month     for o in date_time]
      
  #    num_hours = len(set(all_hours))
  #    num_days = len(set(all_days))
  #    num_months = len(set(all_months))

      all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
      all_days   = np.array(all_days).reshape(len(all_days), 1)
  #    all_months = np.array(all_months).reshape(len(all_months), 1)

      global onehot_hour_of_day
      global onehot_day_of_week
  #    global onehot_month_of_year
      
      onehot_hour_of_day   = OneHotEncoder(sparse=False)
      onehot_day_of_week   = OneHotEncoder(sparse=False)
  #    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)
      print(type(all_hours))
      encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
      encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
  #    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

      encoderX = []
      encoderY = []
      decoderX = []
      decoderY = []

      # For later use, just skip...
      def convertList(inputNumber):
          return [inputNumber]

      # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
      # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
      for i in range(dataSize - T):
          # -------------------- Samples
          customXsamples=[]

          # j -> each look back timestep, index of the timesteps from 0 to M
          for j in range(T):
              # -------------------- Timesteps
              # Add each OneHotEncoding as a feature
              seriesOutputOneHot = []
              if j < M:
                  encoderMode = True
              else:
                  encoderMode = False

              # All features are enabled
              hour_of_day_enabled   = True
              day_of_week_enabled   = True
  #            month_of_year_enabled = True

              # -------------------- Features
              # Add features based on the flags
              global num_hours
              global num_days
  #            global num_months
              if hour_of_day_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                  num_hours = onehot_hour_of_day.get_feature_names().shape[0]

              if day_of_week_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                  num_days = onehot_day_of_week.get_feature_names().shape[0]
  #
  #            if month_of_year_enabled:
  #                # Add for the whole horizon
  #                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
  #                num_months = onehot_month_of_year.get_feature_names().shape[0]



              # Form input array
              add_to_input_array = []
              if encoderMode:
                  customXsamples.append([X[i+j]] + seriesOutputOneHot)
              else:
                  customXsamples.append(seriesOutputOneHot)
              # Add features based on flags - DISABLE ALL : no past features are used in the article
              # if hour_of_day_enabled:
              #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
              # if day_of_week_enabled:
              #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
              # if month_of_year_enabled:
              #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


              # Add to encoder or decoder train data
              if M - 1 == j:
                  encoderX.append(customXsamples)
                  encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                  customXsamples = []
              elif T - 1 == j:
                  decoderX.append(customXsamples)
                  decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                  customXsamples = []



      return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)

  # Taken from test3.py
  def prepareDataTesting(Xa, date_time,SCALER):
      """
      Data Representation :
      [samples, timesteps, features]

      Each timestep :
      [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
      Example :
      [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

      Each sample :
      [ timestep_0, timestep_1, ..., timestep_n ]

      Overall(3D) :
      [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
      """

      dataSize = len(Xa)
      scaler = SCALER
      X = scaler.transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])

  #    global num_hours
  #    global num_days
  #    global num_months
      
      # Prepare features
      all_hours  = [o.hour      for o in date_time]
      all_days   = [o.weekday() for o in date_time]
  #    all_months = [o.month     for o in date_time]
      
  #    num_hours = len(set(all_hours))
  #    num_days = len(set(all_days))
  #    num_months = len(set(all_months))

      all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
      all_days   = np.array(all_days).reshape(len(all_days), 1)
  #    all_months = np.array(all_months).reshape(len(all_months), 1)

      global onehot_hour_of_day
      global onehot_day_of_week
  #    global onehot_month_of_year
      
      onehot_hour_of_day   = OneHotEncoder(sparse=False, categories='auto')
      onehot_day_of_week   = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)

      encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
      encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
  #    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

      encoderX = []
      encoderY = []
      decoderX = []
      decoderY = []

      # For later use, just skip...
      def convertList(inputNumber):
          return [inputNumber]

      # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
      # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
      for i in range(dataSize - T):
          # -------------------- Samples
          customXsamples=[]

          # j -> each look back timestep, index of the timesteps from 0 to M
          for j in range(T):
              # -------------------- Timesteps
              # Add each OneHotEncoding as a feature
              seriesOutputOneHot = []
              if j < M:
                  encoderMode = True
              else:
                  encoderMode = False

              # All features are enabled
              hour_of_day_enabled   = True
              day_of_week_enabled   = True
  #            month_of_year_enabled = True

              # -------------------- Features
              # Add features based on the flags
              global num_hours
              global num_days
  #            global num_months
              if hour_of_day_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                  num_hours = onehot_hour_of_day.get_feature_names().shape[0]

              if day_of_week_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                  num_days = onehot_day_of_week.get_feature_names().shape[0]
  #
  #            if month_of_year_enabled:
  #                # Add for the whole horizon
  #                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
  #                num_months = onehot_month_of_year.get_feature_names().shape[0]



              # Form input array
              add_to_input_array = []
              if encoderMode:
                  customXsamples.append([X[i+j]] + seriesOutputOneHot)
              else:
                  customXsamples.append(seriesOutputOneHot)
              # Add features based on flags - DISABLE ALL : no past features are used in the article
              # if hour_of_day_enabled:
              #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
              # if day_of_week_enabled:
              #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
              # if month_of_year_enabled:
              #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


              # Add to encoder or decoder train data
              if M - 1 == j:
                  encoderX.append(customXsamples)
                  encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                  customXsamples = []
              elif T - 1 == j:
                  decoderX.append(customXsamples)
                  decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                  customXsamples = []



      return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)


  global forecasts

  ############# Run code
  trainData, train_date_time = readData('train')
  testData,  test_date_time  = readData('valid')
  validData,  valid_date_time  = readData('test')

  trainScaler, XtrainEncoder, YtrainEncoder, XtrainDecoder, YtrainDecoder = prepareDataTraining(trainData, train_date_time)
  testScaler,  XtestEncoder,  YtestEncoder,  XtestDecoder,  YtestDecoder  = prepareDataTesting(testData , test_date_time, trainScaler)
  validScaler,  XvalidEncoder,  YvalidEncoder,  XvalidDecoder,  YvalidDecoder  = prepareDataTesting(validData , valid_date_time, trainScaler)


  ### Tensorboard modelling
  logdirEnc = "load_final/encoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  logdirDec = "/content/drive/My Drive/Colab Notebooks/load/decoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboardCallbackEncoder = TensorBoard(log_dir = logdirEnc)
  tensorboardCallbackDecoder = TensorBoard(log_dir = logdirDec)

  ### Create Encoder model

  # Model layers
  num_features_encoder = 1 + (num_months + num_days + num_hours)
  inputEncoder = Input(batch_shape=(batchSize, M, num_features_encoder))
  encoderLSTM = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
  # encoderLSTM2 = LSTM(numNeuron, return_sequences=True, return_state=True, recurrent_dropout=0.4, kernel_regularizer=regularizers.l2(0.000001) ,stateful=stateful,activation = "relu")
  #encoderLSTM3 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
  encoderDense = TimeDistributed(Dense(1))


  # Model connections
  encoderLSTMOutputs, _, _ = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM
  # encoderLSTM2Outputs      = encoderLSTM2(encoderLSTMOutputs)
  # encoderLSTM2Outputs, _, _= encoderLSTM2(encoderLSTMOutputs)
  encoderOutputs           = encoderDense(encoderLSTMOutputs)


  encoderModel = Model(inputs=inputEncoder, outputs=encoderOutputs)
  encoderModel.compile(optimizer='adam', loss='mse')

  encoderFilepath="/content/drive/My Drive/Colab Notebooks/load/models/encoder-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
  encoderCheckpoint = ModelCheckpoint(encoderFilepath, monitor='val_loss', verbose=1, save_best_only=False,save_weights_only=False, mode='min')

  print('Training encoder...'); encoderModel.fit(XtrainEncoder, YtrainEncoder,epochs=encoderEpoch, shuffle=False, validation_data=(XtestEncoder, YtestEncoder),callbacks=[encoderCheckpoint,tensorboardCallbackEncoder])
  # encoderModel = load_model('/content/drive/My Drive/Colab Notebooks/encoder_load_e50.hdf5')


  ### Create merged (Encoder + Decoder) model

  # Model layers
  num_features_decoder   = (num_months + num_days + num_hours)
  inputDecoder           = Input(batch_shape=(batchSize, T-M, num_features_decoder))
  decoderBroadcasted     = Lambda(lambda x: broadcast_to(x, [batchSize, T-M, numNeuron]))
  decoderConcatenated    = Lambda(lambda x: concat(x, axis=2))
  decoderAllZeros        = Lambda(lambda x: tf.zeros(shape =(batchSize, numNeuron)))
  decoderLSTM            = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
  # decoderLSTM2           = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
  #decoderLSTM3           = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
  decoderDense           = TimeDistributed(Dense(1))
  layerV                 = Dense(numNeuron)
  layerVActivation       = Activation('relu')
  layerV_prime           = Dense(numNeuron)
  layerV_primeActivation = Activation('relu')

  # Model connections
  encoderLSTMOutputs, _, encoder_h = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM


  # Map encoder hidden state final to some vector(simple layer)
  c = layerVActivation(layerV(encoder_h))
  # Remap the c vector to h_prime
  h_prime = layerV_primeActivation(layerV_prime(c))

  # Use h_prime as input to other layers
  c_broadcasted              = decoderBroadcasted(c)
  decoderConcatenatedOutputs = decoderConcatenated([inputDecoder, c_broadcasted])

  # decoderStateZeros : just zeros to feed to the internal state
  decoderStateZeros  = decoderAllZeros(encoder_h) # dummy argument
  decoderLSTMOutputs = decoderLSTM(decoderConcatenatedOutputs, initial_state=[decoderStateZeros, h_prime])
  # decoderLSTMOutputs2     = decoderLSTM2(decoderLSTMOutputs)
  #decoderLSTMOutputs3     = decoderLSTM3(decoderLSTMOutputs2)
  decoderOutputs     = decoderDense(decoderLSTMOutputs)


  mergedModel = Model(inputs=[inputEncoder, inputDecoder], outputs=[decoderOutputs])
  mergedModel.compile(optimizer='adam', loss='mse')
  print('Training merged(encoder + decoder) model...');

  outputs = list()

  
  

  for i in range(decoderEpoch):
      decoderFilepath="/content/drive/My Drive/Colab Notebooks/load/models/decoder-weights-improvement-"+str(i)+"-{val_loss:.2f}.hdf5"
      decoderCheckpoint = ModelCheckpoint(decoderFilepath, monitor='val_loss', verbose=1, save_best_only=False,save_weights_only=False, mode='min')
      mergedModel.fit([XtrainEncoder, XtrainDecoder], [YtrainDecoder],epochs=1, shuffle=False, validation_data=([XtestEncoder, XtestDecoder], YtestDecoder),callbacks=[decoderCheckpoint,tensorboardCallbackDecoder])


      lstmOuts = mergedModel.predict([XtrainEncoder, XtrainDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = trainScaler.inverse_transform(lstmOuts)
      plot(i, trainData, lstmOuts, 'train')
      error_train = calculateError(trainData, lstmOuts)

      lstmOuts = mergedModel.predict([XtestEncoder, XtestDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = testScaler.inverse_transform(lstmOuts)
      plot(i, testData, lstmOuts, 'test')
      error_test = calculateError(testData, lstmOuts)
      outputs.append(lstmOuts[-1])
      outputsDF = pd.DataFrame(outputs)
      if i%10 == 0:
          outputsDF.to_csv(directoryName + '/' + 'output_' + str(i) + '.csv', sep = ',')
          outputs = list()
      lstmOuts = mergedModel.predict([XvalidEncoder, XvalidDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = validScaler.inverse_transform(lstmOuts)
      plotTest(i, validData, lstmOuts, 'test')
      
      print('train errors: ', error_train)
      print('test errors: ',error_test)
      

  #np.savetxt('pv_forecasts.txt', forecasts,delimiter=',')

<class 'numpy.ndarray'>


RuntimeError: ignored

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  7 15:07:54 2020

@author: yubim
"""

# Functional API test file


'''
Notes :
  TimeDistributed : create model that process sequence of inputs

'''

'''
Additional notes :
  When encoder is traine completely, merged model exhibits much better
  performance rather than training decoder with more epochs.
'''


'''
Variables :
  encoder_h : h^<N>, encoder hidden state(maps to internal state of LSTM, C)
  decoder_h : decoder hidden state(maps to internal state of LSTM, C)

'''
%tensorflow_version 2.x
import os
from os import listdir
from os.path import isfile, join

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Lambda, Activation, Dropout

import numpy as np
import pandas as pd

import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.metrics import mean_absolute_percentage_error
from tensorflow.keras import regularizers

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Parameters
from tensorflow import concat
from tensorflow import broadcast_to


import tensorflow as tf

import matplotlib

tf.test.gpu_device_name()

with tf.device('/device:GPU:0'):
# Different configurations for different computation structures
# gpu = input('On Server?(y/n) : ')
  gpu = 'n'
  if(gpu == 'y'):
      # Ask for the parameters
      encoderEpoch    = 10
      decoderEpoch    = 5
      numNeuron       = 128
      batchSize       = 1

      forecastHorizon = input('T - M         : ')
      directoryName   = input('Directory     : ') # directory for the output plots (without slash, Example : test10-1)
      dataSize        = input('Data size     : ')
      M               = input('M             : ')


      matplotlib.use('pdf')

      forecastHorizon = int(forecastHorizon)
      M               = int(M)
      dataSize        = int(dataSize)

      T = M + forecastHorizon

      stateful = True


      # sys.stdout = open(directoryName + '/' + 'log.txt', "w")
  else:
      # Ask for the parameters
      encoderEpoch    = 50
      decoderEpoch    = 300
      numNeuron       = 512
      batchSize       = 1

      forecastHorizon = '24'
      directoryName   = '/content/drive/My Drive/Colab Notebooks/load' # directory for the output plots (without slash, Example : test10-1)
      dataSize        = '1440'
      M               = '24'


      matplotlib.use('pdf')

      forecastHorizon = int(forecastHorizon)
      M               = int(M)
      dataSize        = int(dataSize)

      T = M + forecastHorizon



      stateful = False



      os.environ['CUDA_VISIBLE_DEVICES']="-1"

  import matplotlib.pyplot as plt




  #################### Code starts
  # Parameters for LSTM input (we need to know the input shape)
  # These are dynamic (2 month will not yield OneHotEncoding with 12 length)
  num_months = 0
  num_days   = 0
  num_hours  = 0

  def plot(figNum, originalData, lstmOuts, customTag=''):
      # lstmOuts -> predictions inverse transformed
      # originalData -> data read from file

      # forecastLong : forecast with given (T - M)
      forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      listStart = -50
      listEnd   = -2
      plt.plot(originalData[listStart:listEnd])
      plt.plot(forecastLong[listStart:listEnd])
      plt.plot(persis[listStart+1:listEnd+1])
      plt.plot(startingPoints[listStart:listEnd], 'rX')

      title = ''
      title = title + customTag + '\n'
      title = title + ('numNeuron = ' + str(numNeuron) + ', ')
      title = title + ('dataSize = ' + str(dataSize) + ', ')
      title = title + ('M = ' + str(M) + ', ')
      title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
      title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
      # title = title + ('loss = ' + str(loss) + ', ')
      # title = title + ('optimizer = ' + str(optimizer) + ', ')
      title = title + ('(T - M) = ' + str(T-M) + ', ')
      title = title + ('currentEpoch = ' + str(figNum) + '')
      plt.title(title, fontsize=6)

      plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
      plt.legend(['Original Data', 'LSTM', 'Persistance', 'First Forecast'])
      plt.savefig(directoryName + '/' + customTag + str(figNum) + '.png', bbox_inches="tight")
      plt.cla()

  def plotTest(figNum, originalData, lstmOuts, customTag=''):
      # lstmOuts -> predictions inverse transformed
      # originalData -> data read from file

      # forecastLong : forecast with given (T - M)
      forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      listStart = -50
      listEnd   = -2
      plt.plot(originalData[listStart:listEnd])
      plt.plot(forecastLong[listStart:listEnd])
      plt.plot(persis[listStart+1:listEnd+1])
      plt.plot(startingPoints[listStart:listEnd], 'rX')

      title = ''
      title = title + customTag + '\n'
      title = title + ('numNeuron = ' + str(numNeuron) + ', ')
      title = title + ('dataSize = ' + str(dataSize) + ', ')
      title = title + ('M = ' + str(M) + ', ')
      title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
      title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
      # title = title + ('loss = ' + str(loss) + ', ')
      # title = title + ('optimizer = ' + str(optimizer) + ', ')
      title = title + ('(T - M) = ' + str(T-M) + ', ')
      title = title + ('currentEpoch = ' + str(figNum) + '')
      plt.title(title, fontsize=6)

      plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
      plt.legend(['Original Data', 'LSTM', 'Persistance', 'First Forecast'])
      plt.savefig(directoryName + '/' + customTag + str(figNum) + '_test.png', bbox_inches="tight")
      plt.cla()


  def noneNum(allorders):
      # Replace None's in a python list with 0's and return the output
      allorders = list(allorders)
      output = [i if i is not None else 0 for i in allorders]

      return output

  def calculateError(originalData, lstmOuts):
      forecastLong   = [lstmOuts[i//(T - M)*(T - M)][i%(T - M)] for i in range(lstmOuts.shape[0])]
      startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

      forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
      startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
      persis         = np.concatenate([np.array([None]), originalData])

      # Calculate error
      endIndex = -(T - M) + 1
      if endIndex == 0:
          endIndex = originalData.shape[0]

      text = ''

      maemym  = mean_absolute_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      maepers = mean_absolute_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      text = text + "MAE my model     : " + str(maemym)
      text = text + '\n'
      text = text + "MAE persistence  : " + str(maepers)
      text = text + '\n'
      text = text + '\n'

      msemym  = mean_squared_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      msepers = mean_squared_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      text = text + "MSE my model     : " + str(msemym)
      text = text + '\n'
      text = text + "MSE persistence  : " + str(msepers)
      text = text + '\n'
      text = text + '\n'

      # # TODO - can be replaced by tensorflow
      # mapemym  = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      # text = text + "MAPE my model    : " + str(mapemym)
      # text = text + '\n'
      # text = text + "MAPE persistence : " + str(mapepers)
      # text = text + '\n'
      # text = text + '\n'
      
      # mapemym = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
      # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
      # with tensorflow.Session() as sess:
          # mapemym_val = mapemym.eval()
          # mapepers_val = mapepers.eval()
      
      # text = text + "MAPE my model    : " + str(mapemym_val)
      # text = text + '\n'
      # text = text + "MAPE persistence : " + str(mapepers_val)
      # text = text + '\n'
      # text = text + '\n'


      return text

  #def mean_absolute_percentage_error(y_true, y_pred):
  #    # Not implemented in sklearn
  #
  #    # Not recommended to use since negative values
  #    # create negative losses
  #
  #    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  def readData(dataType):
      if dataType == 'train':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_train.csv',sep=',')
      elif dataType == 'valid':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_valid.csv',sep=',')
      elif dataType == 'test':
          data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_test.csv',sep=',')


      outputDat = np.array(data.total_consumption.iloc[:len(data)])
      date_time = [datetime.datetime.strptime(data.timestamp.iloc[i], '%Y-%m-%d %H:%M:%S') for i in range(1,len(data))]

      return outputDat, date_time


  # Taken from test3.py
  # Taken from test3.py
  def prepareDataTraining(Xa, date_time):
      """
      Data Representation :
      [samples, timesteps, features]

      Each timestep :
      [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
      Example :
      [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

      Each sample :
      [ timestep_0, timestep_1, ..., timestep_n ]

      Overall(3D) :
      [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
      """
      
      dataSize = len(Xa)

      scaler = MinMaxScaler(feature_range =(-1,1))
      X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])
      
  #    global num_hours
  #    global num_days
  #    global num_months
      
      # Prepare features
      all_hours  = [o.hour      for o in date_time]
      all_days   = [o.weekday() for o in date_time]
  #    all_months = [o.month     for o in date_time]
      
  #    num_hours = len(set(all_hours))
  #    num_days = len(set(all_days))
  #    num_months = len(set(all_months))

      all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
      all_days   = np.array(all_days).reshape(len(all_days), 1)
  #    all_months = np.array(all_months).reshape(len(all_months), 1)

      global onehot_hour_of_day
      global onehot_day_of_week
  #    global onehot_month_of_year
      
      onehot_hour_of_day   = OneHotEncoder(sparse=False)
      onehot_day_of_week   = OneHotEncoder(sparse=False)
  #    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)
      print(type(all_hours))
      encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
      encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
  #    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

      encoderX = []
      encoderY = []
      decoderX = []
      decoderY = []

      # For later use, just skip...
      def convertList(inputNumber):
          return [inputNumber]

      # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
      # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
      for i in range(dataSize - T):
          # -------------------- Samples
          customXsamples=[]

          # j -> each look back timestep, index of the timesteps from 0 to M
          for j in range(T):
              # -------------------- Timesteps
              # Add each OneHotEncoding as a feature
              seriesOutputOneHot = []
              if j < M:
                  encoderMode = True
              else:
                  encoderMode = False

              # All features are enabled
              hour_of_day_enabled   = True
              day_of_week_enabled   = True
  #            month_of_year_enabled = True

              # -------------------- Features
              # Add features based on the flags
              global num_hours
              global num_days
  #            global num_months
              if hour_of_day_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                  num_hours = onehot_hour_of_day.get_feature_names().shape[0]

              if day_of_week_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                  num_days = onehot_day_of_week.get_feature_names().shape[0]
  #
  #            if month_of_year_enabled:
  #                # Add for the whole horizon
  #                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
  #                num_months = onehot_month_of_year.get_feature_names().shape[0]



              # Form input array
              add_to_input_array = []
              if encoderMode:
                  customXsamples.append([X[i+j]] + seriesOutputOneHot)
              else:
                  customXsamples.append(seriesOutputOneHot)
              # Add features based on flags - DISABLE ALL : no past features are used in the article
              # if hour_of_day_enabled:
              #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
              # if day_of_week_enabled:
              #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
              # if month_of_year_enabled:
              #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


              # Add to encoder or decoder train data
              if M - 1 == j:
                  encoderX.append(customXsamples)
                  encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                  customXsamples = []
              elif T - 1 == j:
                  decoderX.append(customXsamples)
                  decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                  customXsamples = []



      return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)

  # Taken from test3.py
  def prepareDataTesting(Xa, date_time,SCALER):
      """
      Data Representation :
      [samples, timesteps, features]

      Each timestep :
      [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
      Example :
      [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

      Each sample :
      [ timestep_0, timestep_1, ..., timestep_n ]

      Overall(3D) :
      [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
      """

      dataSize = len(Xa)
      scaler = SCALER
      X = scaler.transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])

  #    global num_hours
  #    global num_days
  #    global num_months
      
      # Prepare features
      all_hours  = [o.hour      for o in date_time]
      all_days   = [o.weekday() for o in date_time]
  #    all_months = [o.month     for o in date_time]
      
  #    num_hours = len(set(all_hours))
  #    num_days = len(set(all_days))
  #    num_months = len(set(all_months))

      all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
      all_days   = np.array(all_days).reshape(len(all_days), 1)
  #    all_months = np.array(all_months).reshape(len(all_months), 1)

      global onehot_hour_of_day
      global onehot_day_of_week
  #    global onehot_month_of_year
      
      onehot_hour_of_day   = OneHotEncoder(sparse=False, categories='auto')
      onehot_day_of_week   = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
  #    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)

      encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
      encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
  #    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

      encoderX = []
      encoderY = []
      decoderX = []
      decoderY = []

      # For later use, just skip...
      def convertList(inputNumber):
          return [inputNumber]

      # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
      # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
      for i in range(dataSize - T):
          # -------------------- Samples
          customXsamples=[]

          # j -> each look back timestep, index of the timesteps from 0 to M
          for j in range(T):
              # -------------------- Timesteps
              # Add each OneHotEncoding as a feature
              seriesOutputOneHot = []
              if j < M:
                  encoderMode = True
              else:
                  encoderMode = False

              # All features are enabled
              hour_of_day_enabled   = True
              day_of_week_enabled   = True
  #            month_of_year_enabled = True

              # -------------------- Features
              # Add features based on the flags
              global num_hours
              global num_days
  #            global num_months
              if hour_of_day_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                  num_hours = onehot_hour_of_day.get_feature_names().shape[0]

              if day_of_week_enabled:
                  # Add for the whole horizon
                  seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                  num_days = onehot_day_of_week.get_feature_names().shape[0]
  #
  #            if month_of_year_enabled:
  #                # Add for the whole horizon
  #                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
  #                num_months = onehot_month_of_year.get_feature_names().shape[0]



              # Form input array
              add_to_input_array = []
              if encoderMode:
                  customXsamples.append([X[i+j]] + seriesOutputOneHot)
              else:
                  customXsamples.append(seriesOutputOneHot)
              # Add features based on flags - DISABLE ALL : no past features are used in the article
              # if hour_of_day_enabled:
              #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
              # if day_of_week_enabled:
              #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
              # if month_of_year_enabled:
              #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


              # Add to encoder or decoder train data
              if M - 1 == j:
                  encoderX.append(customXsamples)
                  encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                  customXsamples = []
              elif T - 1 == j:
                  decoderX.append(customXsamples)
                  decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                  customXsamples = []



      return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)


  global forecasts

  ############# Run code
  trainData, train_date_time = readData('train')
  testData,  test_date_time  = readData('valid')
  validData,  valid_date_time  = readData('test')

  trainScaler, XtrainEncoder, YtrainEncoder, XtrainDecoder, YtrainDecoder = prepareDataTraining(trainData, train_date_time)
  testScaler,  XtestEncoder,  YtestEncoder,  XtestDecoder,  YtestDecoder  = prepareDataTesting(testData , test_date_time, trainScaler)
  validScaler,  XvalidEncoder,  YvalidEncoder,  XvalidDecoder,  YvalidDecoder  = prepareDataTesting(validData , valid_date_time, trainScaler)


  ### Tensorboard modelling
  logdirEnc = "load_final/encoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  logdirDec = "/content/drive/My Drive/Colab Notebooks/load/decoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboardCallbackEncoder = TensorBoard(log_dir = logdirEnc)
  tensorboardCallbackDecoder = TensorBoard(log_dir = logdirDec)

  ### Create Encoder model

  # Model layers
  num_features_encoder = 1 + (num_months + num_days + num_hours)
  inputEncoder = Input(batch_shape=(batchSize, M, num_features_encoder))
  encoderLSTM = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
  # encoderLSTM2 = LSTM(numNeuron, return_sequences=True, return_state=True, recurrent_dropout=0.4, kernel_regularizer=regularizers.l2(0.000001) ,stateful=stateful,activation = "relu")
  #encoderLSTM3 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
  encoderDense = TimeDistributed(Dense(1))


  # Model connections
  encoderLSTMOutputs, _, _ = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM
  # encoderLSTM2Outputs      = encoderLSTM2(encoderLSTMOutputs)
  # encoderLSTM2Outputs, _, _= encoderLSTM2(encoderLSTMOutputs)
  encoderOutputs           = encoderDense(encoderLSTMOutputs)


  encoderModel = Model(inputs=inputEncoder, outputs=encoderOutputs)
  encoderModel.compile(optimizer='adam', loss='mse')

  encoderFilepath="/content/drive/My Drive/Colab Notebooks/load/models/encoder-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
  encoderCheckpoint = ModelCheckpoint(encoderFilepath, monitor='val_loss', verbose=1, save_best_only=False,save_weights_only=False, mode='min')

  #print('Training encoder...'); encoderModel.fit(XtrainEncoder, YtrainEncoder,epochs=encoderEpoch, shuffle=False, validation_data=(XtestEncoder, YtestEncoder),callbacks=[encoderCheckpoint,tensorboardCallbackEncoder])
  encoderModel = load_model('/content/drive/My Drive/Colab Notebooks/load_old/models/encoder-weights-improvement-12-0.07.hdf5')


  ### Create merged (Encoder + Decoder) model

  # Model layers
  num_features_decoder   = (num_months + num_days + num_hours)
  inputDecoder           = Input(batch_shape=(batchSize, T-M, num_features_decoder))
  decoderBroadcasted     = Lambda(lambda x: broadcast_to(x, [batchSize, T-M, numNeuron]))
  decoderConcatenated    = Lambda(lambda x: concat(x, axis=2))
  decoderAllZeros        = Lambda(lambda x: tf.zeros(shape =(batchSize, numNeuron)))
  decoderLSTM            = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
  decoderLSTM2           = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
  #decoderLSTM3           = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
  decoderDense           = TimeDistributed(Dense(1))
  layerV                 = Dense(numNeuron)
  layerVActivation       = Activation('relu')
  layerV_prime           = Dense(numNeuron)
  layerV_primeActivation = Activation('relu')

  # Model connections
  encoderLSTMOutputs, _, encoder_h = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM


  # Map encoder hidden state final to some vector(simple layer)
  c = layerVActivation(layerV(encoder_h))
  # Remap the c vector to h_prime
  h_prime = layerV_primeActivation(layerV_prime(c))

  # Use h_prime as input to other layers
  c_broadcasted              = decoderBroadcasted(c)
  decoderConcatenatedOutputs = decoderConcatenated([inputDecoder, c_broadcasted])

  # decoderStateZeros : just zeros to feed to the internal state
  decoderStateZeros  = decoderAllZeros(encoder_h) # dummy argument
  decoderLSTMOutputs = decoderLSTM(decoderConcatenatedOutputs, initial_state=[decoderStateZeros, h_prime])
  decoderLSTMOutputs2     = decoderLSTM2(decoderLSTMOutputs)
  #decoderLSTMOutputs3     = decoderLSTM3(decoderLSTMOutputs2)
  decoderOutputs     = decoderDense(decoderLSTMOutputs2)


  mergedModel = Model(inputs=[inputEncoder, inputDecoder], outputs=[decoderOutputs])
  mergedModel.compile(optimizer='adam', loss='mse')
  print('Training merged(encoder + decoder) model...');

  outputs = list()
  mergedModel = load_model('/content/drive/My Drive/Colab Notebooks/load/models/decoder-weights-improvement-49-0.11.hdf5')



  for i in range(50, decoderEpoch):
      decoderFilepath="/content/drive/My Drive/Colab Notebooks/load/models/decoder-weights-improvement-"+str(i)+"-{val_loss:.2f}.hdf5"
      decoderCheckpoint = ModelCheckpoint(decoderFilepath, monitor='val_loss', verbose=1, save_best_only=False,save_weights_only=False, mode='min')
      mergedModel.fit([XtrainEncoder, XtrainDecoder], [YtrainDecoder],epochs=1, shuffle=False, validation_data=([XtestEncoder, XtestDecoder], YtestDecoder),callbacks=[decoderCheckpoint,tensorboardCallbackDecoder])


      lstmOuts = mergedModel.predict([XtrainEncoder, XtrainDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = trainScaler.inverse_transform(lstmOuts)
      plot(i, trainData, lstmOuts, 'train')
      error_train = calculateError(trainData, lstmOuts)

      lstmOuts = mergedModel.predict([XtestEncoder, XtestDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = testScaler.inverse_transform(lstmOuts)
      plot(i, testData, lstmOuts, 'test')
      error_test = calculateError(testData, lstmOuts)
      outputs.append(lstmOuts[-1])
      outputsDF = pd.DataFrame(outputs)
      if i%10 == 0:
          outputsDF.to_csv(directoryName + '/' + 'output_' + str(i) + '.csv', sep = ',')
          outputs = list()
      lstmOuts = mergedModel.predict([XvalidEncoder, XvalidDecoder])
      lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
      lstmOuts = validScaler.inverse_transform(lstmOuts)
      plotTest(i, validData, lstmOuts, 'test')
      
      print('train errors: ', error_train)
      print('test errors: ',error_test)
      

#np.savetxt('pv_forecasts.txt', forecasts,delimiter=',')

<class 'numpy.ndarray'>
Training merged(encoder + decoder) model...
Train on 1248 samples, validate on 384 samples
1247/1248 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/load/models/decoder-weights-improvement-50-0.13.hdf5
1248/1248 [==============================] - 79s 63ms/sample - loss: 0.0081 - val_loss: 0.1302
train errors:  MAE my model     : 3.2838093596672993
MAE persistence  : 2.519923939151321

MSE my model     : 20.548924574493164
MSE persistence  : 12.194020664631706


test errors:  MAE my model     : 3.891441972172725
MAE persistence  : 2.6011974025974025

MSE my model     : 32.00149426806342
MSE persistence  : 12.116096900324676


Train on 1248 samples, validate on 384 samples
1247/1248 [============================>.] - ETA: 0s - loss: 0.0055
Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/load/models/decoder-weights-improvement-51-0.13.hdf5
1248/1248 [================

ValueError: ignored

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  7 15:07:54 2020

@author: yubim
"""

# Functional API test file


'''
Notes :
    TimeDistributed : create model that process sequence of inputs

'''

'''
Additional notes :
    When encoder is traine completely, merged model exhibits much better
    performance rather than training decoder with more epochs.
'''


'''
Variables :
    encoder_h : h^<N>, encoder hidden state(maps to internal state of LSTM, C)
    decoder_h : decoder hidden state(maps to internal state of LSTM, C)

'''
import os
from os import listdir
from os.path import isfile, join

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Lambda, Activation, Dropout

import numpy as np
import pandas as pd

import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.metrics import mean_absolute_percentage_error

from tensorflow.keras.callbacks import TensorBoard


import tensorflow as tf

# Parameters
from tensorflow import concat
from tensorflow import broadcast_to

import matplotlib

# Different configurations for different computation structures
# gpu = input('On Server?(y/n) : ')
gpu = 'n'
if(gpu == 'y'):
    # Ask for the parameters
    encoderEpoch    = 10
    decoderEpoch    = 5
    numNeuron       = 128
    batchSize       = 1

    forecastHorizon = input('T - M         : ')
    directoryName   = input('Directory     : ') # directory for the output plots (without slash, Example : test10-1)
    dataSize        = input('Data size     : ')
    M               = input('M             : ')


    matplotlib.use('pdf')

    forecastHorizon = int(forecastHorizon)
    M               = int(M)
    dataSize        = int(dataSize)

    T = M + forecastHorizon

    stateful = True


    # sys.stdout = open(directoryName + '/' + 'log.txt', "w")
else:
    # Ask for the parameters
    encoderEpoch    = 100
    decoderEpoch    = 300
    numNeuron       = 512
    batchSize       = 1

    forecastHorizon = '24'
    directoryName   = '/content/drive/My Drive/Colab Notebooks' # directory for the output plots (without slash, Example : test10-1)
    dataSize        = '1440'
    M               = '24'


    matplotlib.use('pdf')

    forecastHorizon = int(forecastHorizon)
    M               = int(M)
    dataSize        = int(dataSize)

    T = M + forecastHorizon



    stateful = False



    os.environ['CUDA_VISIBLE_DEVICES']="-1"

import matplotlib.pyplot as plt




#################### Code starts
# Parameters for LSTM input (we need to know the input shape)
# These are dynamic (2 month will not yield OneHotEncoding with 12 length)
num_months = 0
num_days   = 0
num_hours  = 0

def plot(figNum, originalData, lstmOuts, customTag=''):
    # lstmOuts -> predictions inverse transformed
    # originalData -> data read from file

    # forecastLong : forecast with given (T - M)
    forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
    startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

    forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
    startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
    persis         = np.concatenate([np.array([None]), originalData])

    listStart = -25
    listEnd   = -1
    plt.plot(originalData[listStart:listEnd])
    plt.plot(forecastLong[listStart:listEnd])
#    plt.plot(persis[listStart+1:listEnd+1])
#    plt.plot(startingPoints[listStart:listEnd], 'rX')

    title = ''
    title = title + customTag + '\n'
    title = title + ('numNeuron = ' + str(numNeuron) + ', ')
    title = title + ('dataSize = ' + str(dataSize) + ', ')
    title = title + ('M = ' + str(M) + ', ')
    title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
    title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
    # title = title + ('loss = ' + str(loss) + ', ')
    # title = title + ('optimizer = ' + str(optimizer) + ', ')
    title = title + ('(T - M) = ' + str(T-M) + ', ')
    title = title + ('currentEpoch = ' + str(figNum) + '')
    plt.title(title, fontsize=6)

#    plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
    plt.legend(['Original Data', 'LSTM'])
    plt.savefig(directoryName + '/' + customTag + str(figNum) + 'test.png', bbox_inches="tight")
    plt.cla()
    return forecastLong[listStart:listEnd],originalData[listStart:listEnd]


def noneNum(allorders):
    # Replace None's in a python list with 0's and return the output
    allorders = list(allorders)
    output = [i if i is not None else 0 for i in allorders]

    return output

def calculateError(originalData, lstmOuts):
    forecastLong   = [lstmOuts[i//(T - M)*(T - M)][i%(T - M)] for i in range(lstmOuts.shape[0])]
    startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

    forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
    startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
    persis         = np.concatenate([np.array([None]), originalData])

    # Calculate error
    endIndex = -(T - M) + 1
    if endIndex == 0:
        endIndex = originalData.shape[0]

    text = ''

    maemym  = mean_absolute_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    maepers = mean_absolute_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    text = text + "MAE my model     : " + str(maemym)
    text = text + '\n'
    text = text + "MAE persistence  : " + str(maepers)
    text = text + '\n'
    text = text + '\n'

    msemym  = mean_squared_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    msepers = mean_squared_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    text = text + "MSE my model     : " + str(msemym)
    text = text + '\n'
    text = text + "MSE persistence  : " + str(msepers)
    text = text + '\n'
    text = text + '\n'

    # # TODO - can be replaced by tensorflow
    # mapemym  = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    # text = text + "MAPE my model    : " + str(mapemym)
    # text = text + '\n'
    # text = text + "MAPE persistence : " + str(mapepers)
    # text = text + '\n'
    # text = text + '\n'
    
    # mapemym = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    # with tensorflow.Session() as sess:
        # mapemym_val = mapemym.eval()
        # mapepers_val = mapepers.eval()
    
    # text = text + "MAPE my model    : " + str(mapemym_val)
    # text = text + '\n'
    # text = text + "MAPE persistence : " + str(mapepers_val)
    # text = text + '\n'
    # text = text + '\n'


    return text

#def mean_absolute_percentage_error(y_true, y_pred):
#    # Not implemented in sklearn
#
#    # Not recommended to use since negative values
#    # create negative losses
#
#    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def readData(dataType):
    if dataType == 'train':
        data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_train.csv',sep=',')
    elif dataType == 'test':
        data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_test.csv',sep=',')

    outputDat = np.array(data.total_consumption.iloc[:len(data)])
    date_time = [datetime.datetime.strptime(data.timestamp.iloc[i], '%Y-%m-%d %H:%M:%S') for i in range(1,len(data))]

    return outputDat, date_time


# Taken from test3.py
# Taken from test3.py
def prepareDataTraining(Xa, date_time):
    """
    Data Representation :
    [samples, timesteps, features]

    Each timestep :
    [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
    Example :
    [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

    Each sample :
    [ timestep_0, timestep_1, ..., timestep_n ]

    Overall(3D) :
    [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
    """
    
    dataSize = len(Xa)

    scaler = MinMaxScaler(feature_range =(-1,1))
    X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])
    
#    global num_hours
#    global num_days
#    global num_months
    
    # Prepare features
    all_hours  = [o.hour      for o in date_time]
    all_days   = [o.weekday() for o in date_time]
#    all_months = [o.month     for o in date_time]
    
#    num_hours = len(set(all_hours))
#    num_days = len(set(all_days))
#    num_months = len(set(all_months))

    all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
    all_days   = np.array(all_days).reshape(len(all_days), 1)
#    all_months = np.array(all_months).reshape(len(all_months), 1)

    global onehot_hour_of_day
    global onehot_day_of_week
#    global onehot_month_of_year
    
    onehot_hour_of_day   = OneHotEncoder(sparse=False)
    onehot_day_of_week   = OneHotEncoder(sparse=False)
#    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)
    print(type(all_hours))
    encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
    encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
#    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

    encoderX = []
    encoderY = []
    decoderX = []
    decoderY = []

    # For later use, just skip...
    def convertList(inputNumber):
        return [inputNumber]

    # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
    # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
    for i in range(dataSize - T):
        # -------------------- Samples
        customXsamples=[]

        # j -> each look back timestep, index of the timesteps from 0 to M
        for j in range(T):
            # -------------------- Timesteps
            # Add each OneHotEncoding as a feature
            seriesOutputOneHot = []
            if j < M:
                encoderMode = True
            else:
                encoderMode = False

            # All features are enabled
            hour_of_day_enabled   = True
            day_of_week_enabled   = True
#            month_of_year_enabled = True

            # -------------------- Features
            # Add features based on the flags
            global num_hours
            global num_days
#            global num_months
            if hour_of_day_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                num_hours = onehot_hour_of_day.get_feature_names().shape[0]

            if day_of_week_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                num_days = onehot_day_of_week.get_feature_names().shape[0]
#
#            if month_of_year_enabled:
#                # Add for the whole horizon
#                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
#                num_months = onehot_month_of_year.get_feature_names().shape[0]



            # Form input array
            add_to_input_array = []
            if encoderMode:
                customXsamples.append([X[i+j]] + seriesOutputOneHot)
            else:
                customXsamples.append(seriesOutputOneHot)
            # Add features based on flags - DISABLE ALL : no past features are used in the article
            # if hour_of_day_enabled:
            #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
            # if day_of_week_enabled:
            #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
            # if month_of_year_enabled:
            #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


            # Add to encoder or decoder train data
            if M - 1 == j:
                encoderX.append(customXsamples)
                encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                customXsamples = []
            elif T - 1 == j:
                decoderX.append(customXsamples)
                decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                customXsamples = []



    return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)

# Taken from test3.py
def prepareDataTesting(Xa, date_time):
    """
    Data Representation :
    [samples, timesteps, features]

    Each timestep :
    [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
    Example :
    [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

    Each sample :
    [ timestep_0, timestep_1, ..., timestep_n ]

    Overall(3D) :
    [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
    """

    dataSize = len(Xa)
    scaler = MinMaxScaler(feature_range =(-1,1))
    X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])

#    global num_hours
#    global num_days
#    global num_months
    
    # Prepare features
    all_hours  = [o.hour      for o in date_time]
    all_days   = [o.weekday() for o in date_time]
#    all_months = [o.month     for o in date_time]
    
#    num_hours = len(set(all_hours))
#    num_days = len(set(all_days))
#    num_months = len(set(all_months))

    all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
    all_days   = np.array(all_days).reshape(len(all_days), 1)
#    all_months = np.array(all_months).reshape(len(all_months), 1)

    global onehot_hour_of_day
    global onehot_day_of_week
#    global onehot_month_of_year
    
    onehot_hour_of_day   = OneHotEncoder(sparse=False, categories='auto')
    onehot_day_of_week   = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)

    encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
    encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
#    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

    encoderX = []
    encoderY = []
    decoderX = []
    decoderY = []

    # For later use, just skip...
    def convertList(inputNumber):
        return [inputNumber]

    # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
    # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
    for i in range(dataSize - T):
        # -------------------- Samples
        customXsamples=[]

        # j -> each look back timestep, index of the timesteps from 0 to M
        for j in range(T):
            # -------------------- Timesteps
            # Add each OneHotEncoding as a feature
            seriesOutputOneHot = []
            if j < M:
                encoderMode = True
            else:
                encoderMode = False

            # All features are enabled
            hour_of_day_enabled   = True
            day_of_week_enabled   = True
#            month_of_year_enabled = True

            # -------------------- Features
            # Add features based on the flags
            global num_hours
            global num_days
#            global num_months
            if hour_of_day_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                num_hours = onehot_hour_of_day.get_feature_names().shape[0]

            if day_of_week_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                num_days = onehot_day_of_week.get_feature_names().shape[0]
#
#            if month_of_year_enabled:
#                # Add for the whole horizon
#                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
#                num_months = onehot_month_of_year.get_feature_names().shape[0]



            # Form input array
            add_to_input_array = []
            if encoderMode:
                customXsamples.append([X[i+j]] + seriesOutputOneHot)
            else:
                customXsamples.append(seriesOutputOneHot)
            # Add features based on flags - DISABLE ALL : no past features are used in the article
            # if hour_of_day_enabled:
            #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
            # if day_of_week_enabled:
            #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
            # if month_of_year_enabled:
            #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


            # Add to encoder or decoder train data
            if M - 1 == j:
                encoderX.append(customXsamples)
                encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                customXsamples = []
            elif T - 1 == j:
                decoderX.append(customXsamples)
                decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                customXsamples = []



    return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)


global forecasts

############# Run code
trainData, train_date_time = readData('train')
testData,  test_date_time  = readData('test')

trainScaler, XtrainEncoder, YtrainEncoder, XtrainDecoder, YtrainDecoder = prepareDataTraining(trainData, train_date_time)
testScaler,  XtestEncoder,  YtestEncoder,  XtestDecoder,  YtestDecoder  = prepareDataTesting(testData , test_date_time)


### Tensorboard modelling
#logdirEnc = "pv_final/encoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdirDec = "pv_final" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboardCallbackEncoder = TensorBoard(log_dir = logdirEnc)
tensorboardCallbackDecoder = TensorBoard(log_dir = logdirDec)

### Create Encoder model

# # Model layers
# num_features_encoder = 1 + (num_months + num_days + num_hours)
# inputEncoder = Input(batch_shape=(batchSize, M, num_features_encoder))
# encoderLSTM = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful, activation = "relu")
# #encoderLSTM2 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
# #encoderLSTM3 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
# encoderDense = TimeDistributed(Dense(1))


# # Model connections
# encoderLSTMOutputs, _, _  = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM
# #encoderLSTM2Outputs      = encoderLSTM2(encoderLSTMOutputs)
# #encoderLSTM3Outputs, _, _= encoderLSTM2(encoderLSTM2Outputs)
# encoderOutputs           = encoderDense(encoderLSTMOutputs)


# encoderModel = Model(inputs=inputEncoder, outputs=encoderOutputs)
# encoderModel.compile(optimizer='adam', loss='mse')
#print('Training encoder...'); encoderModel.fit(XtrainEncoder, YtrainEncoder,verbose = 0,epochs=encoderEpoch, shuffle=False, validation_data=(XtestEncoder, YtestEncoder),callbacks=[tensorboardCallbackEncoder])
encoderModel = load_model('/content/drive/My Drive/Colab Notebooks/load_encoder.hdf5')


# ### Create merged (Encoder + Decoder) model

# # Model layers
# num_features_decoder   = (num_months + num_days + num_hours)
# inputDecoder           = Input(batch_shape=(batchSize, T-M, num_features_decoder))
# decoderBroadcasted     = Lambda(lambda x: tf.broadcast_to(x, [batchSize, T-M, numNeuron]))
# decoderConcatenated    = Lambda(lambda x: tf.concat(x, axis=2))
# decoderAllZeros        = Lambda(lambda x: tf.zeros(shape =(batchSize, numNeuron)))
# decoderLSTM            = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
# decoderLSTM2           = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
# #decoderLSTM3           = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
# decoderDense           = TimeDistributed(Dense(1))
# layerV                 = Dense(numNeuron)
# layerVActivation       = Activation('relu')
# layerV_prime           = Dense(numNeuron)
# layerV_primeActivation = Activation('relu')

# # Model connections
# encoderLSTMOutputs, _, encoder_h = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM


# # Map encoder hidden state final to some vector(simple layer)
# c = layerVActivation(layerV(encoder_h))
# # Remap the c vector to h_prime
# h_prime = layerV_primeActivation(layerV_prime(c))

# # Use h_prime as input to other layers
# c_broadcasted              = decoderBroadcasted(c)
# decoderConcatenatedOutputs = decoderConcatenated([inputDecoder, c_broadcasted])

# # decoderStateZeros : just zeros to feed to the internal state
# decoderStateZeros  = decoderAllZeros(encoder_h) # dummy argument
# decoderLSTMOutputs = decoderLSTM(decoderConcatenatedOutputs, initial_state=[decoderStateZeros, h_prime])
# decoderLSTMOutputs2     = decoderLSTM2(decoderLSTMOutputs)
# #decoderLSTMOutputs3     = decoderLSTM3(decoderLSTMOutputs2)
# decoderOutputs     = decoderDense(decoderLSTMOutputs2)

i = 67
# mergedModel = Model(inputs=[inputEncoder, inputDecoder], outputs=[decoderOutputs])
# mergedModel.compile(optimizer='adam', loss='mse')
print('Training merged(encoder + decoder) model...');
mergedModel = load_model('/content/drive/My Drive/Colab Notebooks/load_decoder.hdf5')
#outputs = list()
# mypath = 'load_decoder/'
# modelList = [f for f in listdir(mypath) if isfile(join(mypath, f))]
foCa = list()
orDa = list()
#    mergedModel = load_model('decoder_load_e86.hdf5')
lstmOuts = mergedModel.predict([XtestEncoder, XtestDecoder])
#lstmOuts = mergedModel.predict([XtrainEncoder, XtrainDecoder])
lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
#    Tscaler = MinMaxScaler(feature_range =(-1,1))
#    lstmOuts = lstmOuts.reshape(lstmOuts.shape[0], lstmOuts.shape[1])
lstmOuts = trainScaler.inverse_transform(lstmOuts)
fc, od = plot(i, testData, lstmOuts, 'test')
#fc, od = plot(i, trainData, lstmOuts, 'test')

foCa.append(fc)
orDa.append(od)

    
    #error_test = calculateError(testData, lstmOuts)
    #outputs.append(lstmOuts[-1])
    #outputsDF = pd.DataFrame(outputs)
    #if i%10 == 0:
    #    outputsDF.to_csv(directoryName + '/' + 'output_' + str(i) + '.csv', sep = ',')
    #    outputs = list()
    
#    print('train errors: ', error_train)
#    print('test errors: ',error_test)
        
#np.savetxt('pv_forecasts.txt', forecasts,delimiter=',')

<class 'numpy.ndarray'>
Training merged(encoder + decoder) model...


In [0]:
fC = pd.DataFrame([od,fc]).T
fC.columns = ['real_values', 'forecasts']
fC.to_csv('load_forecasts.csv',sep = ',')

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  7 15:07:54 2020

@author: yubim
"""

# Functional API test file


'''
Notes :
  TimeDistributed : create model that process sequence of inputs

'''

'''
Additional notes :
  When encoder is traine completely, merged model exhibits much better
  performance rather than training decoder with more epochs.
'''


'''
Variables :
  encoder_h : h^<N>, encoder hidden state(maps to internal state of LSTM, C)
  decoder_h : decoder hidden state(maps to internal state of LSTM, C)

'''
%tensorflow_version 2.x
import os
from os import listdir
from os.path import isfile, join

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Lambda, Activation, Dropout

import numpy as np
import pandas as pd

import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.metrics import mean_absolute_percentage_error

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Parameters
from tensorflow import concat
from tensorflow import broadcast_to


import tensorflow as tf

import matplotlib

# tf.test.gpu_device_name()

# with tf.device('/device:GPU:0'):
# Different configurations for different computation structures
# gpu = input('On Server?(y/n) : ')
gpu = 'n'
if(gpu == 'y'):
    # Ask for the parameters
    encoderEpoch    = 10
    decoderEpoch    = 5
    numNeuron       = 128
    batchSize       = 1

    forecastHorizon = input('T - M         : ')
    directoryName   = input('Directory     : ') # directory for the output plots (without slash, Example : test10-1)
    dataSize        = input('Data size     : ')
    M               = input('M             : ')


    matplotlib.use('pdf')

    forecastHorizon = int(forecastHorizon)
    M               = int(M)
    dataSize        = int(dataSize)

    T = M + forecastHorizon

    stateful = True


    # sys.stdout = open(directoryName + '/' + 'log.txt', "w")
else:
    # Ask for the parameters
    encoderEpoch    = 50
    decoderEpoch    = 300
    numNeuron       = 512
    batchSize       = 1

    forecastHorizon = '24'
    directoryName   = '/content/drive/My Drive/Colab Notebooks/load' # directory for the output plots (without slash, Example : test10-1)
    dataSize        = '1440'
    M               = '24'


    matplotlib.use('pdf')

    forecastHorizon = int(forecastHorizon)
    M               = int(M)
    dataSize        = int(dataSize)

    T = M + forecastHorizon



    stateful = False



    os.environ['CUDA_VISIBLE_DEVICES']="-1"

import matplotlib.pyplot as plt




#################### Code starts
# Parameters for LSTM input (we need to know the input shape)
# These are dynamic (2 month will not yield OneHotEncoding with 12 length)
num_months = 0
num_days   = 0
num_hours  = 0

def plot(figNum, originalData, lstmOuts, customTag=''):
    # lstmOuts -> predictions inverse transformed
    # originalData -> data read from file

    # forecastLong : forecast with given (T - M)
    forecastLong   = [lstmOuts[i//(T-M)*(T-M)][i%(T-M)] for i in range(lstmOuts.shape[0])]
    startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

    forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
    startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
    persis         = np.concatenate([np.array([None]), originalData])

    listStart = -50
    listEnd   = -2
    plt.plot(originalData[listStart:listEnd])
    plt.plot(forecastLong[listStart:listEnd])
    plt.plot(persis[listStart+1:listEnd+1])
    plt.plot(startingPoints[listStart:listEnd], 'rX')

    title = ''
    title = title + customTag + '\n'
    title = title + ('numNeuron = ' + str(numNeuron) + ', ')
    title = title + ('dataSize = ' + str(dataSize) + ', ')
    title = title + ('M = ' + str(M) + ', ')
    title = title + ('encoderEpoch = ' + str(encoderEpoch) + ', ')
    title = title + ('decoderEpoch = ' + str(decoderEpoch) + ', \n')
    # title = title + ('loss = ' + str(loss) + ', ')
    # title = title + ('optimizer = ' + str(optimizer) + ', ')
    title = title + ('(T - M) = ' + str(T-M) + ', ')
    title = title + ('currentEpoch = ' + str(figNum) + '')
    plt.title(title, fontsize=6)

    plt.text(-0.1, 0, calculateError(originalData, lstmOuts), fontsize=6)
    plt.legend(['Original Data', 'LSTM', 'Persistance', 'First Forecast'])
    plt.savefig(directoryName + '/' + customTag + str(figNum) + '.png', bbox_inches="tight")
    plt.cla()


def noneNum(allorders):
    # Replace None's in a python list with 0's and return the output
    allorders = list(allorders)
    output = [i if i is not None else 0 for i in allorders]

    return output

def calculateError(originalData, lstmOuts):
    forecastLong   = [lstmOuts[i//(T - M)*(T - M)][i%(T - M)] for i in range(lstmOuts.shape[0])]
    startingPoints = [lstmOuts[i][0] for i in range(lstmOuts.shape[0])]

    forecastLong   = np.concatenate([np.array([None]*(M+1)), forecastLong])
    startingPoints = np.concatenate([np.array([None]*(M+1)), startingPoints])
    persis         = np.concatenate([np.array([None]), originalData])

    # Calculate error
    endIndex = -(T - M) + 1
    if endIndex == 0:
        endIndex = originalData.shape[0]

    text = ''

    maemym  = mean_absolute_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    maepers = mean_absolute_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    text = text + "MAE my model     : " + str(maemym)
    text = text + '\n'
    text = text + "MAE persistence  : " + str(maepers)
    text = text + '\n'
    text = text + '\n'

    msemym  = mean_squared_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    msepers = mean_squared_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    text = text + "MSE my model     : " + str(msemym)
    text = text + '\n'
    text = text + "MSE persistence  : " + str(msepers)
    text = text + '\n'
    text = text + '\n'

    # # TODO - can be replaced by tensorflow
    # mapemym  = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    # text = text + "MAPE my model    : " + str(mapemym)
    # text = text + '\n'
    # text = text + "MAPE persistence : " + str(mapepers)
    # text = text + '\n'
    # text = text + '\n'
    
    # mapemym = mean_absolute_percentage_error(noneNum(forecastLong[M:]) , originalData[M:endIndex])
    # mapepers = mean_absolute_percentage_error(noneNum(persis[M:-(T - M)]), originalData[M:endIndex])
    # with tensorflow.Session() as sess:
        # mapemym_val = mapemym.eval()
        # mapepers_val = mapepers.eval()
    
    # text = text + "MAPE my model    : " + str(mapemym_val)
    # text = text + '\n'
    # text = text + "MAPE persistence : " + str(mapepers_val)
    # text = text + '\n'
    # text = text + '\n'


    return text

#def mean_absolute_percentage_error(y_true, y_pred):
#    # Not implemented in sklearn
#
#    # Not recommended to use since negative values
#    # create negative losses
#
#    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def readData(dataType):
    # if dataType == 'train':
    #     data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_train.csv',sep=',')
    # elif dataType == 'valid':
    data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/load_valid.csv',sep=',')

    outputDat = np.array(data.total_consumption.iloc[:len(data)])
    date_time = [datetime.datetime.strptime(data.timestamp.iloc[i], '%Y-%m-%d %H:%M:%S') for i in range(1,len(data))]

    return outputDat, date_time


# Taken from test3.py
# Taken from test3.py
def prepareDataTraining(Xa, date_time):
    """
    Data Representation :
    [samples, timesteps, features]

    Each timestep :
    [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
    Example :
    [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

    Each sample :
    [ timestep_0, timestep_1, ..., timestep_n ]

    Overall(3D) :
    [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
    """
    
    dataSize = len(Xa)

    scaler = MinMaxScaler(feature_range =(-1,1))
    X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])
    
#    global num_hours
#    global num_days
#    global num_months
    
    # Prepare features
    all_hours  = [o.hour      for o in date_time]
    all_days   = [o.weekday() for o in date_time]
#    all_months = [o.month     for o in date_time]
    
#    num_hours = len(set(all_hours))
#    num_days = len(set(all_days))
#    num_months = len(set(all_months))

    all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
    all_days   = np.array(all_days).reshape(len(all_days), 1)
#    all_months = np.array(all_months).reshape(len(all_months), 1)

    global onehot_hour_of_day
    global onehot_day_of_week
#    global onehot_month_of_year
    
    onehot_hour_of_day   = OneHotEncoder(sparse=False)
    onehot_day_of_week   = OneHotEncoder(sparse=False)
#    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)
    print(type(all_hours))
    encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
    encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
#    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

    encoderX = []
    encoderY = []
    decoderX = []
    decoderY = []

    # For later use, just skip...
    def convertList(inputNumber):
        return [inputNumber]

    # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
    # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
    for i in range(dataSize - T):
        # -------------------- Samples
        customXsamples=[]

        # j -> each look back timestep, index of the timesteps from 0 to M
        for j in range(T):
            # -------------------- Timesteps
            # Add each OneHotEncoding as a feature
            seriesOutputOneHot = []
            if j < M:
                encoderMode = True
            else:
                encoderMode = False

            # All features are enabled
            hour_of_day_enabled   = True
            day_of_week_enabled   = True
#            month_of_year_enabled = True

            # -------------------- Features
            # Add features based on the flags
            global num_hours
            global num_days
#            global num_months
            if hour_of_day_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                num_hours = onehot_hour_of_day.get_feature_names().shape[0]

            if day_of_week_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                num_days = onehot_day_of_week.get_feature_names().shape[0]
#
#            if month_of_year_enabled:
#                # Add for the whole horizon
#                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
#                num_months = onehot_month_of_year.get_feature_names().shape[0]



            # Form input array
            add_to_input_array = []
            if encoderMode:
                customXsamples.append([X[i+j]] + seriesOutputOneHot)
            else:
                customXsamples.append(seriesOutputOneHot)
            # Add features based on flags - DISABLE ALL : no past features are used in the article
            # if hour_of_day_enabled:
            #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
            # if day_of_week_enabled:
            #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
            # if month_of_year_enabled:
            #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


            # Add to encoder or decoder train data
            if M - 1 == j:
                encoderX.append(customXsamples)
                encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                customXsamples = []
            elif T - 1 == j:
                decoderX.append(customXsamples)
                decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                customXsamples = []



    return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)

# Taken from test3.py
def prepareDataTesting(Xa, date_time):
    """
    Data Representation :
    [samples, timesteps, features]

    Each timestep :
    [consumptionValue, 24 vector of One Hot Encoding of Input, forecast * 24 vector of One Hot Encoding of Output]
    Example :
    [0.13, 0, 0, 1, 0, ..., 0, | 0, 1, ..., 0, | 0, 0, 0, 1, .., 0 ] -> 1 + 24 + np.rizon * 24 elements (| is used as a seperator to make it readeable)

    Each sample :
    [ timestep_0, timestep_1, ..., timestep_n ]

    Overall(3D) :
    [ [ timestep_0, timestep_1, ..., timestep_n ], [ timestep_0, timestep_1, ..., timestep_n ] ... ]
    """

    dataSize = len(Xa)
    scaler = MinMaxScaler(feature_range =(-1,1))
    X = scaler.fit_transform(Xa.reshape(Xa.shape[0],1)).reshape(Xa.shape[0])

#    global num_hours
#    global num_days
#    global num_months
    
    # Prepare features
    all_hours  = [o.hour      for o in date_time]
    all_days   = [o.weekday() for o in date_time]
#    all_months = [o.month     for o in date_time]
    
#    num_hours = len(set(all_hours))
#    num_days = len(set(all_days))
#    num_months = len(set(all_months))

    all_hours  = np.array(all_hours).reshape(len(all_hours), 1)
    all_days   = np.array(all_days).reshape(len(all_days), 1)
#    all_months = np.array(all_months).reshape(len(all_months), 1)

    global onehot_hour_of_day
    global onehot_day_of_week
#    global onehot_month_of_year
    
    onehot_hour_of_day   = OneHotEncoder(sparse=False, categories='auto')
    onehot_day_of_week   = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(sparse=False, categories='auto')
#    onehot_month_of_year = OneHotEncoder(n_values = 13, sparse=False)

    encoded_hour_of_day   = onehot_hour_of_day.fit_transform(all_hours)
    encoded_day_of_week   = onehot_day_of_week.fit_transform(all_days)
#    encoded_month_of_year = onehot_month_of_year.fit_transform(all_months)

    encoderX = []
    encoderY = []
    decoderX = []
    decoderY = []

    # For later use, just skip...
    def convertList(inputNumber):
        return [inputNumber]

    # i -> each sample, index of the samples from 0 to (ds - M - (T - M) + 1) = (ds - T + 1)
    # TODO - sanki + 1 olmamali asagida??? -> olunca disina tasti zaten, bir gariplik var ama cozemedim, eski kod ile farki ne ki buranin?
    for i in range(dataSize - T):
        # -------------------- Samples
        customXsamples=[]

        # j -> each look back timestep, index of the timesteps from 0 to M
        for j in range(T):
            # -------------------- Timesteps
            # Add each OneHotEncoding as a feature
            seriesOutputOneHot = []
            if j < M:
                encoderMode = True
            else:
                encoderMode = False

            # All features are enabled
            hour_of_day_enabled   = True
            day_of_week_enabled   = True
#            month_of_year_enabled = True

            # -------------------- Features
            # Add features based on the flags
            global num_hours
            global num_days
#            global num_months
            if hour_of_day_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_hour_of_day[i+j])
                num_hours = onehot_hour_of_day.get_feature_names().shape[0]

            if day_of_week_enabled:
                # Add for the whole horizon
                seriesOutputOneHot.extend(encoded_day_of_week[i+j])
                num_days = onehot_day_of_week.get_feature_names().shape[0]
#
#            if month_of_year_enabled:
#                # Add for the whole horizon
#                seriesOutputOneHot.extend(encoded_month_of_year[i+j])
#                num_months = onehot_month_of_year.get_feature_names().shape[0]



            # Form input array
            add_to_input_array = []
            if encoderMode:
                customXsamples.append([X[i+j]] + seriesOutputOneHot)
            else:
                customXsamples.append(seriesOutputOneHot)
            # Add features based on flags - DISABLE ALL : no past features are used in the article
            # if hour_of_day_enabled:
            #     add_to_input_array.extend(list(encoded_hour_of_day[i+j]))
            # if day_of_week_enabled:
            #     add_to_input_array.extend(list(encoded_day_of_week[i+j]))
            # if month_of_year_enabled:
            #     add_to_input_array.extend(list(encoded_month_of_year[i+j]))


            # Add to encoder or decoder train data
            if M - 1 == j:
                encoderX.append(customXsamples)
                encoderY.append(list(map(convertList, X[i + 1 : i + M + 1])))
                customXsamples = []
            elif T - 1 == j:
                decoderX.append(customXsamples)
                decoderY.append(list(map(convertList, X[i + M + 1 : i + T + 1])))
                customXsamples = []



    return scaler, np.array(encoderX), np.array(encoderY), np.array(decoderX), np.array(decoderY)


global forecasts

############# Run code
# trainData, train_date_time = readData('train')
testData,  test_date_time  = readData('valid')

# trainScaler, XtrainEncoder, YtrainEncoder, XtrainDecoder, YtrainDecoder = prepareDataTraining(trainData, train_date_time)
testScaler,  XtestEncoder,  YtestEncoder,  XtestDecoder,  YtestDecoder  = prepareDataTesting(testData , test_date_time)



### Tensorboard modelling
# logdirEnc = "load_final/encoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# logdirDec = "/content/drive/My Drive/Colab Notebooks/load/decoder/logs/tests/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboardCallbackEncoder = TensorBoard(log_dir = logdirEnc)
# tensorboardCallbackDecoder = TensorBoard(log_dir = logdirDec)

### Create Encoder model

# Model layers
num_features_encoder = 1 + (num_months + num_days + num_hours)
inputEncoder = Input(batch_shape=(batchSize, M, num_features_encoder))
encoderLSTM = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful, activation = "relu")
encoderLSTM2 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
#encoderLSTM3 = LSTM(numNeuron, return_sequences=True, return_state=True, stateful=stateful,activation = "relu")
encoderDense = TimeDistributed(Dense(1))


# Model connections
encoderLSTMOutputs  = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM
encoderLSTM2Outputs, _, _      = encoderLSTM2(encoderLSTMOutputs)
#encoderLSTM3Outputs, _, _= encoderLSTM2(encoderLSTM2Outputs)
encoderOutputs           = encoderDense(encoderLSTM2Outputs)


encoderModel = Model(inputs=inputEncoder, outputs=encoderOutputs)
encoderModel.compile(optimizer='adam', loss='mse')

# encoderFilepath="/content/drive/My Drive/Colab Notebooks/load/models/encoder-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
# encoderCheckpoint = ModelCheckpoint(encoderFilepath, monitor='val_loss', verbose=1, save_best_only=False,save_weights_only=False, mode='min')

# print('Training encoder...'); encoderModel.fit(XtrainEncoder, YtrainEncoder,epochs=encoderEpoch, shuffle=False, validation_data=(XtestEncoder, YtestEncoder),callbacks=[encoderCheckpoint,tensorboardCallbackEncoder])
encoderModel = load_model('/content/drive/My Drive/Colab Notebooks/encoder_load_e31.hdf5')


### Create merged (Encoder + Decoder) model

# Model layers
num_features_decoder   = (num_months + num_days + num_hours)
inputDecoder           = Input(batch_shape=(batchSize, T-M, num_features_decoder))
decoderBroadcasted     = Lambda(lambda x: broadcast_to(x, [batchSize, T-M, numNeuron]))
decoderConcatenated    = Lambda(lambda x: concat(x, axis=2))
decoderAllZeros        = Lambda(lambda x: tf.zeros(shape =(batchSize, numNeuron)))
decoderLSTM            = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
decoderLSTM2           = LSTM(numNeuron, return_sequences =True,stateful=stateful,dropout = 0.5,activation="relu")
#decoderLSTM3           = LSTM(numNeuron, return_sequences =True,stateful=stateful,activation="relu")
decoderDense           = TimeDistributed(Dense(1))
layerV                 = Dense(numNeuron)
layerVActivation       = Activation('relu')
layerV_prime           = Dense(numNeuron)
layerV_primeActivation = Activation('relu')

# Model connections
encoderLSTMOutputs, _, encoder_h = encoderLSTM(inputEncoder) #encoders_c is already equal to encoderLSTM


# Map encoder hidden state final to some vector(simple layer)
c = layerVActivation(layerV(encoder_h))
# Remap the c vector to h_prime
h_prime = layerV_primeActivation(layerV_prime(c))

# Use h_prime as input to other layers
c_broadcasted              = decoderBroadcasted(c)
decoderConcatenatedOutputs = decoderConcatenated([inputDecoder, c_broadcasted])

# decoderStateZeros : just zeros to feed to the internal state
decoderStateZeros  = decoderAllZeros(encoder_h) # dummy argument
decoderLSTMOutputs = decoderLSTM(decoderConcatenatedOutputs, initial_state=[decoderStateZeros, h_prime])
decoderLSTMOutputs2     = decoderLSTM2(decoderLSTMOutputs)
#decoderLSTMOutputs3     = decoderLSTM3(decoderLSTMOutputs2)
decoderOutputs     = decoderDense(decoderLSTMOutputs2)


mergedModel = Model(inputs=[inputEncoder, inputDecoder], outputs=[decoderOutputs])
mergedModel.compile(optimizer='adam', loss='mse')
print('Training merged(encoder + decoder) model...');

outputs = list()

MSEs = list()  
Y = np.reshape(YtestDecoder,(384,24))
mypath = 'drive/My Drive/Colab Notebooks/load_encoder_e31/models/'
modelList = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# losses = [float(modelList[i][-9:-5]) for i in range(len(modelList))]
# losses = pd.DataFrame(losses,columns=['mse'])
# best = list(losses.sort_values(by=['mse']).index[0:30])
for i in range(len(modelList)):
  mod = str(mypath + modelList[i])
  mergedModel = load_model(mod)
  lstmOuts = mergedModel.predict([XtestEncoder, XtestDecoder])
  lstmOuts = np.reshape(lstmOuts,(lstmOuts.shape[0],lstmOuts.shape[1]))
  
  e = mean_squared_error(Y[383], lstmOuts[383])
  print('epoch ' + str(i) + ': ',e)
  MSEs.append(e)

    

#np.savetxt('pv_forecasts.txt', forecasts,delimiter=',')

Training merged(encoder + decoder) model...
epoch 0:  0.10467874044809904
epoch 1:  0.10112596166952374
epoch 2:  0.11784212822555158
epoch 3:  0.1015719222055224
epoch 4:  0.10621091395238376
epoch 5:  0.11772072235405845
epoch 6:  0.13293850888003997
epoch 7:  0.13189223309580347
epoch 8:  0.11358428743143467
epoch 9:  0.08226117662382378
epoch 10:  0.13985049914032077
epoch 11:  0.14165580250722026
epoch 12:  0.12765024949081058
epoch 13:  0.127611663903006
epoch 14:  0.14529340865305485
epoch 15:  0.12266158304554214
epoch 16:  0.17156123007753343
epoch 17:  0.18232842117102563
epoch 18:  0.2165286736222031
epoch 19:  0.13522620823628362
epoch 20:  0.20055130042187885
epoch 21:  0.18562296880282356
epoch 22:  0.3661199011685519
epoch 23:  0.3066160275142061
epoch 24:  0.2636105299340961
epoch 25:  0.31446538672954016
epoch 26:  0.27044288950565226
epoch 27:  0.22968294239885342
epoch 28:  0.2555471259578157
epoch 29:  0.28822726660745007
epoch 30:  0.30203351005962775
epoch 31:  0.

In [0]:
len(range(25,73))

(384, 24)

In [0]:
from os import listdir
from os.path import isfile, join
import pandas as pd
mypath = 'drive/My Drive/Colab Notebooks/load_encoder_e31/models/'
modelList = [f for f in listdir(mypath) if isfile(join(mypath, f))]
losses = [float(modelList[i][-9:-5]) for i in range(len(modelList))]
losses = pd.DataFrame(losses,columns=['mse'])
best = list(losses.sort_values(by=['mse']).index[0:30])

In [0]:
MSE = pd.DataFrame(MSEs, columns=['mse'],index = [list(losses.sort_values(by=['mse']).index[0:30])])
MSE.sort_values(by=['mse'])

,mse
0,0.079869
1,0.082763
8,0.083405
3,0.090285
2,0.091217
9,0.094732
4,0.094818
12,0.095690
6,0.096684
5,0.096790


In [0]:
best

[0,
 1,
 8,
 2,
 3,
 4,
 9,
 19,
 17,
 14,
 13,
 12,
 6,
 5,
 10,
 15,
 16,
 51,
 48,
 94,
 70,
 69,
 52,
 92,
 36,
 98,
 21,
 18,
 24,
 71]